In [2]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
import torch.nn.functional as F
from datasets import load_dataset
import matplotlib.pyplot as plt
import pandas as pd

/opt/miniconda3/envs/DL-Project/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: 'dlopen(/opt/miniconda3/envs/DL-Project/lib/python3.10/site-packages/torchvision/image.so, 0x0006): Symbol not found: __ZN3c1017RegisterOperatorsD1Ev
  Referenced from: <9B280146-BBD7-3F77-9873-F9740F2A5329> /opt/miniconda3/envs/DL-project/lib/python3.10/site-packages/torchvision/image.so
  Expected in:     <650A7408-F84B-37A3-B9D0-8E799E9E0543> /opt/miniconda3/envs/DL-project/lib/python3.10/site-packages/torch/lib/libtorch_cpu.dylib'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


### Cifar10 dataset 

In [3]:
cifar = load_dataset("cifar10",)
i = cifar["train"]["img"][0]
print(i.size)

(32, 32)


Classes extraction

In [4]:
labels = cifar["train"].features["label"].names
label2id, id2label = dict(), dict()
for i, label in enumerate(labels):
    label2id[label] = str(i)
    id2label[str(i)] = label
    
print(labels)

['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']


### Loading trained models

In [35]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


In [36]:
model_names = ['resnet50', 'vgg16', 'densenet121']
models_to_compare = {}

for name in model_names:
    model = getattr(models, name)(pretrained=True)
    models_to_compare[name] = model

/opt/miniconda3/envs/DL-Project/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/miniconda3/envs/DL-Project/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
/opt/miniconda3/envs/DL-Project/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weigh

In [37]:
models_to_compare["resnet50"]

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [38]:
models_to_compare["vgg16"]

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [40]:
def modify_model(model, num_classes):
    # Freeze all layers
    for param in model.parameters():
        param.requires_grad = False

    # Modify the final layer
    if isinstance(model, models.ResNet):
        num_ftrs = model.fc.out_features
        model.fc = nn.Linear(num_ftrs, num_classes)
    elif isinstance(model, models.VGG) or isinstance(model, models.DenseNet):
        num_ftrs = model.classifier[0].in_features
        model.classifier = nn.Sequential(
            nn.Linear(num_ftrs, 256),
            nn.ReLU(),
            nn.Dropout(0.4),
            nn.Linear(256, num_classes)
        )
    else:
        raise NotImplementedError("Model architecture not supported")

    return model

for name in models_to_compare:
    models_to_compare[name] = modify_model(models_to_compare[name], 10)

TypeError: 'Linear' object is not subscriptable

### Dataset Class

In [12]:
transform = transforms.Compose([
    # you can add other transformations in this list
     transforms.Resize((32,32)),
    transforms.ToTensor()
])

class Dataset(Dataset):
    
    # Constructor 
    def __init__(self, X_data, Y_data, transform=transform):
        self.len = len(X_data)
        self.x = X_data
        self.y = Y_data
        self.transform = transform
             
    # Getter
    def __getitem__(self, index):
        x = self.x[index] 
        y = self.y[index]
        if self.transform:
            x = self.transform(x)     
        return x, y
    
    # Get Length
    def __len__(self):
        return self.len


In [13]:
dataset_train = Dataset( X_data = cifar["train"]["img"], Y_data = cifar["train"]["label"])
train_loader = DataLoader(dataset=dataset_train, batch_size=128, shuffle=True)
dataset_test = Dataset( X_data = cifar["test"]["img"], Y_data = cifar["test"]["label"])
test_loader = DataLoader(dataset=dataset_test, batch_size=128, shuffle=True)

Choosing the Optimizer and the Cost function

In [27]:
optimizer = torch.optim.Adam(model.fc.parameters(), lr=0.001)
loss_new = nn.CrossEntropyLoss()

### Training the model

In [43]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()

def get_optimizer(model):
    # Only parameters of final layers are being optimized
    return optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=0.001)

def get_scheduler(optimizer):
    return optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

In [42]:
def accuracy(outputs, labels):
    _, preds = torch.max(outputs, dim=1)
    return torch.tensor(torch.sum(preds == labels).item() / len(preds))


model.train()
for epoch in range(10):

    train_losses = []
    train_acc = []
 
    for x,y in train_loader:

        optimizer.zero_grad()
        x = x.to("cuda:0")
        y = y.to("cuda:0")
        output = model(x)
        loss = loss_new(output, y)
        acc = accuracy(output, y)
        
        loss.backward()
        optimizer.step()

        train_losses.append(loss.item())
        train_acc.append(acc.item())
        

    print("epoch loss", sum(train_losses)/len(train_losses), "acc", sum(train_acc)/len(train_acc))



epoch loss 1.2422582228165453 acc 0.5449688298927854
epoch loss 1.002661668278677 acc 0.6383751598770356
epoch loss 0.8286447400022345 acc 0.7053148976677214
epoch loss 0.699113958936823 acc 0.7522977941176471
epoch loss 0.585098624305652 acc 0.7917798913043478
epoch loss 0.48789337544185124 acc 0.8274536445317671
epoch loss 0.3894337900459309 acc 0.8626918158567775
epoch loss 0.31607526079620546 acc 0.8888666879795396
epoch loss 0.24061696930690799 acc 0.9149216751918159
epoch loss 0.18771342065213892 acc 0.934099264766859


#### Evaluating the model on the test data

In [43]:
model.eval()
with torch.no_grad():
    
    test_losses = []
    test_acc = []
    
    for x,y in test_loader:
        
        x = x.to("cuda:0")
        y = y.to("cuda:0")
        output = model(x)
        
        loss = loss_new(output, y)
        acc = accuracy(output, y)
        
        test_losses.append(loss.item())
        test_acc.append(acc.item())

    print("epoch loss", sum(test_losses)/len(test_losses), "acc", sum(test_acc)/len(test_acc))

epoch loss 0.9411083656021312 acc 0.7643393987341772


Saving the model

In [ ]:
torch.save(model.state_dict(), 'model.pth')